<a href="https://colab.research.google.com/github/anshul-raman/stock_price_prediction/blob/master/Data_mining_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import requests

url = ('http://newsapi.org/v2/everything?'
       'q=Google&'
       'from=2020-10-26&'
       'sortBy=popularity&'
       'apiKey=08927173808245098cb8127ff4c80f77')

response = requests.get(url)

print(response.json())

{'status': 'error', 'code': 'unexpectedError', 'message': "If you're seeing this it means something happened that we weren't expecting. Please try again. If you continue to see this message please email us on support@newsapi.org for help."}


In [3]:
# install ktrain on Google Colab
!pip3 install ktrain
# import ktrain and the ktrain.text modules
import ktrain
from ktrain import text

     |████████████████████████████████| 25.3MB 129kB/s 
     |████████████████████████████████| 983kB 40.0MB/s 
     |████████████████████████████████| 266kB 45.5MB/s 
     |████████████████████████████████| 1.3MB 42.7MB/s 
     |████████████████████████████████| 471kB 45.7MB/s 
     |████████████████████████████████| 1.1MB 42.4MB/s 
     |████████████████████████████████| 2.9MB 42.7MB/s 
     |████████████████████████████████| 890kB 39.5MB/s 
  Created wheel for ktrain: filename=ktrain-0.23.1-cp36-none-any.whl size=25271896 sha256=9be19d819df0a4ba1e9ea5a976030a5939329dfca438fc6d4817720240a0281e
  Stored in directory: /root/.cache/pip/wheels/48/13/4f/fa051ad74b07760a6f4807bac542e18081676937da6f885a0d
  Created wheel for keras-bert: filename=keras_bert-0.86.0-cp36-none-any.whl size=34145 sha256=4c08b49c134d4459f006bc62e10c76cd71ed127ed4a4d2b8e28cde4ff755a5ea
  Stored in directory: /root/.cache/pip/wheels/66/f0/b1/748128b58562fc9e31b907bb5e2ab6a35eb37695e83911236b
  Created wheel for lan

In [4]:
import pandas as pd
import numpy as np
import json
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [5]:
import os
import sys
import bs4
import csv 
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt



import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding,GRU,Dense,Conv2D,MaxPool2D,Input,Dropout,Activation,Bidirectional,Concatenate,Lambda,Reshape,Conv1D,MaxPooling1D,LSTM
from tensorflow.keras.utils import to_categorical
import tensorflow.keras.backend as K
import numpy as np

# ***QUANT DATA***

In [ ]:

GOOG_QUANT_DATA_FILE= pd.read_csv('/content/drive/My Drive/DATA_MINING/stock_price_prediction/data/data/price/Quant/GOOG.csv')
GOOG_QUANT_DATA_FILE_LIST=(list(GOOG_QUANT_DATA_FILE['Date']))



Selecting and normalizing

In [ ]:
GOOG_QUANT_DATA_FILE=GOOG_QUANT_DATA_FILE[['Date','Open','High','Close','Low']]
print(GOOG_QUANT_DATA_FILE)
def normalizer(col):
  scaler=MinMaxScaler(feature_range=(0,1))
  df1=scaler.fit_transform(np.array(col).reshape(-1,1))
  return df1

GOOG_QUANT_DATA_FILE['Open']= normalizer(GOOG_QUANT_DATA_FILE['Open'])
GOOG_QUANT_DATA_FILE['Close']= normalizer(GOOG_QUANT_DATA_FILE['Close'])
GOOG_QUANT_DATA_FILE['High']= normalizer(GOOG_QUANT_DATA_FILE['High'])
GOOG_QUANT_DATA_FILE['Low']= normalizer(GOOG_QUANT_DATA_FILE['Low'])
loc_start_date= 0
for i in range(0,len(GOOG_QUANT_DATA_FILE_LIST)):
  if GOOG_QUANT_DATA_FILE_LIST[i]>=START_DATE:
    loc_start_date=i
    break




print(GOOG_QUANT_DATA_FILE_LIST[loc_start_date],loc_start_date)

   


           Date        Open        High       Close         Low
0    2013-07-01  441.569885  444.374390  442.282227  440.847595
1    2013-07-02  443.437897  443.836395  439.507629  436.997040
2    2013-07-03  438.307129  442.924805  441.559937  437.609741
3    2013-07-05  443.343231  446.033173  445.076752  442.013214
4    2013-07-08  447.926056  451.442871  450.855072  446.865051
..          ...         ...         ...         ...         ...
627  2015-12-24  749.549988  751.349976  748.400024  746.619995
628  2015-12-28  752.919983  762.989990  762.510010  749.520020
629  2015-12-29  766.690002  779.979980  776.599976  766.429993
630  2015-12-30  776.599976  777.599976  771.000000  766.900024
631  2015-12-31  769.500000  769.500000  758.880005  758.340027

[632 rows x 5 columns]
2014-01-02 128


# RAW NEWS DATA ***COLLECTION***

In [ ]:


GOOG_DATA_dir = os.listdir('/content/drive/My Drive/DATA_MINING/stock_price_prediction/data/data/tweet/preprocessed/GOOG')
GOOG_DATA_dir=sorted(GOOG_DATA_dir)
print((GOOG_DATA_dir))

['2014-01-01', '2014-01-02', '2014-01-03', '2014-01-04', '2014-01-05', '2014-01-06', '2014-01-07', '2014-01-08', '2014-01-09', '2014-01-10', '2014-01-11', '2014-01-12', '2014-01-13', '2014-01-14', '2014-01-15', '2014-01-16', '2014-01-19', '2014-01-20', '2014-01-21', '2014-01-22', '2014-01-23', '2014-01-24', '2014-01-25', '2014-01-26', '2014-01-27', '2014-01-28', '2014-01-29', '2014-01-30', '2014-01-31', '2014-02-01', '2014-02-02', '2014-02-03', '2014-02-04', '2014-02-05', '2014-02-06', '2014-02-07', '2014-02-08', '2014-02-09', '2014-02-10', '2014-02-11', '2014-02-12', '2014-02-13', '2014-02-14', '2014-02-15', '2014-02-16', '2014-02-17', '2014-02-18', '2014-02-19', '2014-02-20', '2014-02-21', '2014-02-22', '2014-02-23', '2014-02-24', '2014-02-25', '2014-02-26', '2014-02-27', '2014-02-28', '2014-03-01', '2014-03-02', '2014-03-03', '2014-03-04', '2014-03-05', '2014-03-06', '2014-03-07', '2014-03-08', '2014-03-09', '2014-03-10', '2014-03-11', '2014-03-12', '2014-03-13', '2014-03-14', '2014

In [ ]:
START_DATE=GOOG_DATA_dir[0]
END_DATE=GOOG_DATA_dir[-1]
print(START_DATE,END_DATE)

2014-01-01 2015-12-31


In [ ]:
GOOG_DATA_files = ['/content/drive/My Drive/DATA_MINING/stock_price_prediction/data/data/tweet/preprocessed/GOOG/' + x for x in GOOG_QUANT_DATA_FILE_LIST]
GOOG_DATA_files_1 = ['/content/drive/My Drive/DATA_MINING/stock_price_prediction/data/data/tweet/preprocessed/GOOG/' + x for x in GOOG_DATA_dir]

In [ ]:
def part_preprocess(fileName):
  sentences=[]
  if fileName in GOOG_DATA_files_1:
    f = open(fileName,'r').readlines()
  else :
    return sentences
  print(fileName)
  
  for i in f:
    i=json.loads(i)
    sentences.append(i['text'])
  Output = {} 
 
  for lis in sentences: 
      Output.setdefault(tuple(lis), list()).append(1) 


  for a, b in Output.items(): 
      Output[a] = sum(b)  
  sentences=list(Output.keys())    
  return sentences 


# ***Preprocessing***

In [ ]:
feature_word_data_GOOG=[]

for num in range(0,len(GOOG_DATA_files)):
  feature_word_data_GOOG.append(part_preprocess(GOOG_DATA_files[num]))

print(feature_word_data_GOOG)


/content/drive/My Drive/DATA_MINING/stock_price_prediction/data/data/tweet/preprocessed/GOOG/2014-01-02
/content/drive/My Drive/DATA_MINING/stock_price_prediction/data/data/tweet/preprocessed/GOOG/2014-01-03
/content/drive/My Drive/DATA_MINING/stock_price_prediction/data/data/tweet/preprocessed/GOOG/2014-01-06
/content/drive/My Drive/DATA_MINING/stock_price_prediction/data/data/tweet/preprocessed/GOOG/2014-01-07
/content/drive/My Drive/DATA_MINING/stock_price_prediction/data/data/tweet/preprocessed/GOOG/2014-01-08
/content/drive/My Drive/DATA_MINING/stock_price_prediction/data/data/tweet/preprocessed/GOOG/2014-01-09
/content/drive/My Drive/DATA_MINING/stock_price_prediction/data/data/tweet/preprocessed/GOOG/2014-01-10
/content/drive/My Drive/DATA_MINING/stock_price_prediction/data/data/tweet/preprocessed/GOOG/2014-01-13
/content/drive/My Drive/DATA_MINING/stock_price_prediction/data/data/tweet/preprocessed/GOOG/2014-01-14
/content/drive/My Drive/DATA_MINING/stock_price_prediction/data/

In [ ]:
# l =max(feature_word_data_GOOG,key=len)
# for i in l:
#   print(i)

In [ ]:

contractions = { 
"ain't": "am not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he's": "he is",
"how'd": "how did",
"how'll": "how will",
"how's": "how is",
"i'd": "i would",
"i'll": "i will",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'll": "it will",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"must've": "must have",
"mustn't": "must not",
"needn't": "need not",
"oughtn't": "ought not",
"shan't": "shall not",
"sha'n't": "shall not",
"she'd": "she would",
"she'll": "she will",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"that'd": "that would",
"that's": "that is",
"there'd": "there had",
"there's": "there is",
"they'd": "they would",
"they'll": "they will",
"they're": "they are",
"they've": "they have",
"wasn't": "was not",
"we'd": "we would",
"we'll": "we will",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"where'd": "where did",
"where's": "where is",
"who'll": "who will",
"who's": "who is",
"won't": "will not",
"wouldn't": "would not",
"you'd": "you would",
"you'll": "you will",
"you're": "you are"
}

In [ ]:

stops = set(stopwords.words("english"))     
            
print(stops)
def remove_stops(row):
    my_list = row

    meaningful_words = [w for w in my_list if not w in stops]

    return (meaningful_words)
def preprocess_tweet(news):
    processed_tweet = []
    tweet=news
    if True:
       
        new_text = []
        for word in tweet:
            if word in contractions:
                new_text.append(contractions[word])
            else:
                new_text.append(word)
        tweet = " ".join(new_text)
 
    # Convert to lower case
    tweet = tweet.lower()
    tweet=  re.sub(r"\W", " ",tweet)
    tweet=  re.sub(r"\d", " ",tweet)
    tweet=  re.sub(r"-", " ",tweet)   
    tweet=  re.sub(r"_", " ",tweet)  
   
    # Replaces URLs with the word URL
    tweet = re.sub(r'((www\.[\S]+)|(https?://[\S]+))', ' URL ', tweet)
    # Replace @handle with the word USER_MENTION
    tweet = re.sub(r'@[\S]+', 'USER_MENTION', tweet)
    # Replaces #hashtag with hashtag
    tweet = re.sub(r'#(\S+)', r' \1 ', tweet)
    tweet = re.sub(r'-(\S+)', r' \1 ', tweet)
    # Remove RT (retweet)
    tweet = re.sub(r'\brt\b', '', tweet)
    tweet = re.sub(r'\$', '', tweet)
    # Replace 2+ dots with space
    tweet = re.sub(r'\.{2,}', ' ', tweet)
    # Strip space, " and ' from tweet
    tweet = tweet.strip(' "\'')
    # Replace multiple spaces with a single space
    tweet = re.sub(r'\s+', ' ', tweet)

    words = tweet.split()
    # print(tweet)


    return words




{'their', 'an', 'if', 'than', 'having', 'no', 'ain', 'itself', 'as', 'same', 've', 'myself', 'some', 'o', 'mightn', "shouldn't", 'me', 'i', 'did', 'not', 'will', 'now', 'those', 'while', 'don', 'then', 'shouldn', 'these', 'are', 'we', 'again', 'after', 'aren', 'hadn', 'have', 'this', 'above', 's', 'is', 'too', 'because', 'he', 'that', 'my', 'ours', 'it', 'won', 'was', 'themselves', 'further', 'her', 'there', 'own', 'doing', 'between', 'on', 'y', "hasn't", "isn't", 'd', 'your', 'other', 'off', 'wasn', 'both', 'down', 'hasn', "hadn't", "mustn't", 're', 'just', "you've", "aren't", 'such', 'and', 'you', "you'll", "you're", 'any', 'had', 'where', "you'd", "needn't", 'below', 'but', "doesn't", "that'll", 'yours', 'should', 'him', 'until', 'couldn', 'herself', "mightn't", 'once', 'himself', 'isn', 'here', 'mustn', "haven't", 'yourself', 'theirs', 'weren', 'needn', 'been', 'his', 't', 'doesn', 'does', 'before', 'll', 'to', "shan't", 'of', 'be', 'or', 'over', 'against', 'when', 'they', 'she', '

In [ ]:
pre_features_GOOG=[]
for day in feature_word_data_GOOG:
  temp=[]
  for news in day:
    news=preprocess_tweet(news)
    news=remove_stops(news)
    
    temp=temp+news
  pre_features_GOOG.append(temp) 
 



In [ ]:
print((pre_features_GOOG[128]))
# l= min(pre_features_GOOG,key=len)
# print(len(l))
# print(l)
# l=0
# for i in pre_features_GOOG:
#   print(len(i))

['goog', 'google', 'inc', 'patent', 'issued', 'adding', 'new', 'attributes', 'structured', 'presentation', 'url', 'trade', 'signals', 'generated', 'twitter', 'stream', 'win', 'rate', 'still', 'open', 'goog', 'bidu', 'yelp', 'url', 'goog', 'apple', 'unlock', 'value', 'emulating', 'google', 'way', 'url', 'stock', 'stocks', 'stockaction']


# ***WORD TOKENIZATION***

In [ ]:
word_tokenizer = Tokenizer(oov_token="<OOV>")
word_tokenizer.fit_on_texts(pre_features_GOOG)
word_index = word_tokenizer.word_index
print(len(word_index))
print(word_index)
train_word_features_GOOG = word_tokenizer.texts_to_sequences(pre_features_GOOG)
padded_word_train_features_GOOG = pad_sequences(train_word_features_GOOG ,padding='post',maxlen=300,truncating="post")
print(padded_word_train_features_GOOG.shape)

7575
{'<OOV>': 1, 'goog': 2, 'url': 3, 'user': 4, 'google': 5, 'aapl': 6, 'fb': 7, 'stock': 8, 'stocks': 9, 'googl': 10, 'amzn': 11, 'inc': 12, 'twtr': 13, 'tsla': 14, 'msft': 15, 'nflx': 16, 'qqq': 17, 'new': 18, 'apple': 19, 'p': 20, 'research': 21, 'buy': 22, 'market': 23, 'news': 24, 'c': 25, 'pcln': 26, 'gpro': 27, 'like': 28, 'spy': 29, 'nasdaq': 30, 'x': 31, 'today': 32, 'tech': 33, 'get': 34, 'investing': 35, 'baba': 36, 'bidu': 37, 'via': 38, 'trading': 39, 'alphabet': 40, 'see': 41, 'stockaction': 42, 'sell': 43, 'hft': 44, 'watch': 45, 'video': 46, 'earnings': 47, 'next': 48, 'algos': 49, 'b': 50, 'sigma': 51, 'crossfinder': 52, 'android': 53, 'facebook': 54, 'could': 55, 'time': 56, 'yhoo': 57, 'price': 58, 'great': 59, 'us': 60, 'update': 61, 'ibm': 62, 'ats': 63, 'lx': 64, 'big': 65, 'mobile': 66, 'last': 67, 'k': 68, 'looking': 69, 'penny': 70, 'options': 71, 'lnkd': 72, 'free': 73, 'company': 74, 'high': 75, 'quant': 76, 'billion': 77, 'invest': 78, 'twitter': 79, 'shar

# ***CHAR TOKENIZATION***

In [ ]:
def char_featurizer(pos_feature_dis):
  char_feature = []
  for i in range(len(pos_feature_dis)):
    for j in range(min(len(pos_feature_dis[i]),300)):
      char_feature.append(list(pos_feature_dis[i][j]))
      
    
      
    if len(pos_feature_dis[i]) < 300 :
      if(len(pos_feature_dis[i])<=300):
        for j in range(300-len(pos_feature_dis[i])):
          char_feature.append([])
     

  return (char_feature)

pre_char_features_GOOG= char_featurizer(pre_features_GOOG)



In [ ]:
char_tokenizer = Tokenizer(oov_token="<OOV>")

char_tokenizer.fit_on_texts(pre_char_features_GOOG)

char_index = char_tokenizer.word_index
CHAR_MAX_LENGTH =15

train_charword_sequences = char_tokenizer.texts_to_sequences(pre_char_features_GOOG)
padded_charword_features_GOOG = pad_sequences(train_charword_sequences,padding='post',maxlen=CHAR_MAX_LENGTH,truncating="post")
print(padded_charword_features_GOOG.shape)

padded_charword_features_GOOG= padded_charword_features_GOOG.reshape(-1,300,CHAR_MAX_LENGTH)
print(padded_charword_features_GOOG.shape)

(189600, 15)
(632, 300, 15)


# ***CREATING QUANT DATA SETS***

In [ ]:
def create_dataset_quant(pos_start,dataset, time_step=1,output_time_step=14):
	dataX, dataY = [], []
	for i in range(pos_start,len(dataset)-output_time_step):
		a = dataset[(i-time_step):i]   ###i=0, 0,1,2,3-----99   100 
		dataX.append(a)
		dataY.append(dataset[i][:])
	return np.array(dataX), np.array(dataY)


#important params
input_time_step=100 
output_time_step=1
QUANT_DATA= np.array(GOOG_QUANT_DATA_FILE[['Open','Close','High','Low']])
QUANT_DATA_INPUT ,QUANT_DATA_OUT=create_dataset_quant(loc_start_date,QUANT_DATA,input_time_step,output_time_step)

In [ ]:
def create_dataset_sentiment(pos_start,dataset, time_step=1,output_time_step=14):
  dataY = []
  for i in range(pos_start,len(dataset)-output_time_step):
    change = []
    count=0
    for j in range(0,4):
      if(dataset[i][0]>=dataset[i-1][0]):
        count=count+1
    if(count>=2):
      change.append([1,0])
    else :
      change.append([0,1])   

    dataY.append(change)
  return np.array(dataY)


In [ ]:
DATA_SENTIMENT_ANALYSIS= create_dataset_sentiment(loc_start_date,QUANT_DATA,input_time_step,output_time_step)

In [ ]:
print(DATA_SENTIMENT_ANALYSIS)

[[[1 0]]

 [[0 1]]

 [[0 1]]

 ...

 [[1 0]]

 [[1 0]]

 [[1 0]]]


# ***CREATING NEWS DATA SET***

In [ ]:
news_input_chardata= padded_charword_features_GOOG[loc_start_date:-output_time_step]
news_input_worddata= padded_word_train_features_GOOG[loc_start_date:-output_time_step]


print(news_input_chardata.shape,news_input_worddata.shape,QUANT_DATA_INPUT.shape,QUANT_DATA_OUT.shape,DATA_SENTIMENT_ANALYSIS[200])


(503, 300, 15) (503, 300) (503, 100, 4) (503, 4) [[1 0]]


# TRAIN TEST ***DATA***

# ***MODEL***

In [ ]:
from tensorflow.keras.layers import Flatten,GRUCell
from tensorflow.keras.layers import GRU,LSTM
import numpy as np
from tensorflow.keras.models import Model

from tensorflow.keras.layers import RepeatVector,concatenate,Dot,Lambda
from tensorflow.keras.layers import Reshape


In [ ]:
def pretrained_embedding_layer(word_to_vec_map1,word_to_vec_map2,word_to_index):
  vocab_len = len(word_to_index) + 1
  # emb_dim1 = word_to_vec_map1["और"].shape[0]
  # emb_dim2 = word_to_vec_map2["और"].shape[0]
  emb_matrix = np.zeros((vocab_len,100))

  embedding_layer = Embedding(vocab_len,100,input_length=300)

  
  return embedding_layer

In [ ]:

def charcnn(char_index,block):
    vocab_size = len(char_index)+1
    conv_layers = [[64,3,3],
                   [128,3,-1]]
    dropout_p = 0.3
    num_of_classes = 16
    embedding_size = 30

    fully_connected_layers = [128, 64]
    #print(block)
   # char_sequence = char_vectorizer.texts_to_sequences(block)
  #  char_padded_sequence = pad_sequences(char_sequences,maxlen=CHAR_MAX_LENGTH,padding='post',truncating='post')           
    
    embedding_layer = Embedding(vocab_size,embedding_size,input_length=CHAR_MAX_LENGTH)
  
    x = embedding_layer(block)
    for filter_num, filter_size, pooling_size in conv_layers:
        x = Conv1D(filter_num, filter_size)(x)
        x = Activation('relu')(x)
        if pooling_size != -1:
            x = MaxPooling1D(pool_size=pooling_size)(x)  
    x = Flatten()(x)
    for dense_size in fully_connected_layers:
        x = Dense(dense_size, activation='relu')(x)
        x = Dropout(dropout_p)(x)
    # Output Layer
    predictions = Dense(num_of_classes, activation='relu')(x)
    return predictions

In [ ]:
def mix_model(word_to_vec_map, word_to_vec_map1, word_to_index,input_time_step):
  
  input_shape = (input_time_step,4)

  input= Input(shape=input_shape,dtype='float32',name='input')
  #############embedding##############
  input_shape1 = (300,)
  input_shape2 = (300,15)

  sentence_indices = Input(shape=input_shape1,dtype='int32',name='input1')
  embedding_layer = pretrained_embedding_layer(word_to_vec_map,word_to_vec_map1 ,word_to_index)
  embeddings = embedding_layer(sentence_indices)

  sentence_word = Input(shape=input_shape2,name='input2')
  char_feat = []
  for t in range(300):
    sentence_word_x = Lambda(lambda x : x[:,t,:])(sentence_word)
#    sentence_word_x = Reshape((1,))(sentence_word_x)
    char_feat.append(charcnn(char_index,sentence_word_x))

  char_feat_tensor = Lambda(lambda x : K.stack(x,axis=1))(char_feat)

  mixed_embeddings = Concatenate()([embeddings,char_feat_tensor])
  #####################sentiment analysis###############################
  X1 = (LSTM(units=64,return_sequences=True))(embeddings)

  X1 = Dropout(rate=0.5)(X1)
  ################################################################
  X2 = (LSTM(units=64,return_sequences=True))(input)

  X2 = Dropout(rate=0.5)(X2)
  ################################################################
  X=Concatenate(axis=1)([X2,X1])
  X = (LSTM(units=64,return_sequences=True))(X)
  X = Dropout(rate=0.5)(X)

  X = (LSTM(units=30))(X)
  X = Dropout(rate=0.2)(X)
  print(X.shape)
 

  X = Dense(units=4,activation='sigmoid')(X)

  model = Model(inputs=[input,sentence_indices,sentence_word],outputs=X)
  return model

# New section

In [ ]:

def SA_model(word_to_vec_map, word_to_vec_map1, word_to_index,input_time_step):
  
  #############embedding##############
  input_shape1 = (300,)
  input_shape2 = (300,15)

  sentence_indices = Input(shape=input_shape1,dtype='int32')
  embedding_layer = pretrained_embedding_layer(word_to_vec_map,word_to_vec_map1 ,word_to_index)
  embeddings = embedding_layer(sentence_indices)

  sentence_word = Input(shape=input_shape2)
  char_feat = []
  for t in range(300):
    sentence_word_x = Lambda(lambda x : x[:,t,:])(sentence_word)
#    sentence_word_x = Reshape((1,))(sentence_word_x)
    char_feat.append(charcnn(char_index,sentence_word_x))
  print(len(char_feat))
  print(char_feat[0].shape)
  print(embeddings.shape)
  char_feat_tensor = Lambda(lambda x : K.stack(x,axis=1))(char_feat)
  print(char_feat_tensor.shape)

  
  mixed_embeddings = Concatenate()([embeddings,char_feat_tensor])
  #####################sentiment analysis###############################

  X1 = Bidirectional(LSTM(units=64,return_sequences=True))(mixed_embeddings)

  X1 = Dropout(rate=0.5)(X1)

  ################################################################
 
  X = Bidirectional(LSTM(units=64))(X1)
  X = Dropout(rate=0.5)(X)

  X = Dense(units=4,activation='sigmoid')(X)

  model = Model(inputs=[sentence_indices,sentence_word],outputs=X)
  return model


In [ ]:
def base_model_with_sa(input_time_step):
  
  input_shape = (input_time_step,4)

  input= Input(shape=input_shape,dtype='float32',name='input')
  
  input_shape1 = (4)

  input1 = Input(shape=input_shape1,dtype='float32',name='input1')


  print(input.shape,input1.shape)
  
  X = Bidirectional(LSTM(units=64,return_sequences=True))(input)
  X = Dropout(rate=0.5)(X)
  print(X.shape)
  X = Bidirectional(LSTM(units=30))(X)

  X = Concatenate(axis=-1)([X,input1])
  # X= Reshape((-1,1))(X)
  # X = (LSTM(units=30,return_sequences=True))(X)
  # print(X.shape)
  X = Dropout(rate=0.2)(X)
  print(X.shape)
 

  X = Dense(units=4,activation='sigmoid')(X)

  print(X.shape)

  model = Model(inputs=[input,input1],outputs=X)
  return model

In [ ]:
def base_model(input_time_step):
  
  input_shape = (input_time_step,4)

  input= Input(shape=input_shape,dtype='float32',name='input')
 
  X = Bidirectional(LSTM(units=64,return_sequences=True))(input)


  X = Dropout(rate=0.5)(X)
  print(X.shape)
  X = Bidirectional(LSTM(units=30))(X)
  # X= Reshape((-1,1))(X)
  # X = (LSTM(units=30,return_sequences=True))(X)
  # print(X.shape)
  X = Dropout(rate=0.2)(X)
  print(X.shape)
 

  X = Dense(units=4,activation='sigmoid')(X)

  print(X.shape)

  model = Model(inputs=[input],outputs=X)
  return model

In [ ]:
from tensorflow.keras.optimizers import Adam
opt = Adam(lr=0.004, beta_1=0.9, beta_2=0.999, decay=0.007)
model_base = base_model(input_time_step)
model_base.compile(optimizer=opt, loss='mean_squared_error') 



(None, 100, 128)
(None, 60)
(None, 4)


In [ ]:
from tensorflow.keras.optimizers import Adam
opt = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, decay=0.007)
model_quant_SA = base_model_with_sa(input_time_step)
model_quant_SA.compile(optimizer=opt,loss='mean_squared_error') 



(None, 100, 4) (None, 4)
(None, 100, 128)
(None, 64)
(None, 4)


In [ ]:
from tensorflow.keras.optimizers import Adam

from tensorflow.keras.callbacks import EarlyStopping
opt = Adam(lr=0.004, beta_1=0.9, beta_2=0.999, decay=0.007)
model = mix_model({},{},word_index, input_time_step)
model.compile(optimizer=opt, loss='mean_squared_error') 

print(model.summary())

(None, 30)
Model: "functional_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input1 (InputLayer)             [(None, 300)]        0                                            
__________________________________________________________________________________________________
input (InputLayer)              [(None, 100, 4)]     0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 300, 100)     757600      input1[0][0]                     
__________________________________________________________________________________________________
lstm_5 (LSTM)                   (None, 100, 64)      17664       input[0][0]                      
____________________________________________________________________________

In [ ]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping


opt = Adam(lr=0.004, beta_1=0.9, beta_2=0.999, decay=0.007)
model_SA = SA_model({},{},word_index, input_time_step)
model_SA.compile(optimizer=opt, loss='mse',metrics="accuracy") 

print(model_SA.summary())

Streaming output truncated to the last 5000 lines.
conv1d_725 (Conv1D)             (None, 2, 128)       24704       max_pooling1d_362[0][0]          
__________________________________________________________________________________________________
conv1d_727 (Conv1D)             (None, 2, 128)       24704       max_pooling1d_363[0][0]          
__________________________________________________________________________________________________
conv1d_729 (Conv1D)             (None, 2, 128)       24704       max_pooling1d_364[0][0]          
__________________________________________________________________________________________________
conv1d_731 (Conv1D)             (None, 2, 128)       24704       max_pooling1d_365[0][0]          
__________________________________________________________________________________________________
conv1d_733 (Conv1D)             (None, 2, 128)       24704       max_pooling1d_366[0][0]          
__________________________________________________________

In [ ]:
#######sentiment
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=15)
history=model_SA.fit([news_input_worddata,news_input_chardata],DATA_SENTIMENT_ANALYSIS,validation_split=0.2,epochs=80,batch_size=32,callbacks=[es])

Epoch 1/80


In [ ]:
model_SA.evaluate([news_input_worddata,news_input_chardata],DATA_SENTIMENT_ANALYSIS)

In [ ]:

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=15)
history=model_quant_SA.fit([QUANT_DATA_INPUT,DATA_SENTIMENT_ANALYSIS],QUANT_DATA_OUT,validation_split=0.2,epochs=80,batch_size=32,callbacks=[es])

In [ ]:

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=15)
history=model.fit([QUANT_DATA_INPUT,news_input_worddata],QUANT_DATA_OUT,validation_split=0.2,epochs=80,batch_size=32,callbacks=[es])

In [ ]:

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=15)
model_base.fit(QUANT_DATA_INPUT,QUANT_DATA_OUT,validation_split=0.2,epochs =80,batch_size=32,callbacks=[es])

In [ ]:
model_base.evaluate(QUANT_DATA_INPUT,QUANT_DATA_OUT)0model_quant_SA.evaluate([QUANT_DATA_INPUT,DATA_SENTI0ENT_ANALYSIS],QUANT_DATA_OUT)

# ***Testing area***

In [6]:
NEWS_DATA=pd.read_csv('/content/drive/My Drive/DATA_MINING/stock_price_prediction/data/data/New_data/Combined_News_DJIA.csv')
labels=np.array(NEWS_DATA['Label'])
print(len(labels))
def convert_to_preprocessed(NEWS_DATA):
  labels=np.array(NEWS_DATA['Label'])
  data=[]
  data1=[]
  for i in range(1,26):
    index=("Top"+str(i))
    data.append(np.array(NEWS_DATA[index]))
  print(len(data[0]))  
  for i in range(0,len(labels)):
    s=""
    for j in range(0,25):
      s=s+str(data[j][i])+' '
      print(data[j][i])
    data1.append(s)
  return labels,data1


1989


In [7]:
y_input,x_input=convert_to_preprocessed(NEWS_DATA);

Streaming output truncated to the last 5000 lines.
Unesco clears way for Palestinian membership vote with the US voting against!
Brutal eviction of Palestinian family in Jaffa caught on tape
Showing how futile Internet censorship can be, The Pirate Bay registered new domain name to allow Belgian users to access the site and bypass a recent court order that requires ISPs Belgacom and Telenet to initiate DNS blockades of 11 domains connected to The Pirate Bay
New Danish Minister of the Economy &amp; Interior favors bicycles over freeways - "I rather invest in cycle tracks than freeways." -  In Copenhagen, 80% of commuters already bicycle to work every day, even in winter. 
Footage offers glimpse of North Korean hunger crisis:  Occasionally the veil is pulled back from the secretive state of North Korea and the world gets a glimpse of what life is like there.
Billionaire hedge fund manager George Soros has lost a case at the European Court of Human Rights to have his criminal conviction f

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



David Cameron says Europe must get better at sending migrants home
Eight Million Tons of Plastic Dumped in Ocean Every Year
A civil lawsuit against Volkswagen was filed Tuesday in Sao Paulo accusing the company of allowing the torture and detention of employees opposed to Brazil's former military dictatorship
Diesel "worse than the black plague" and responsible for 50,000 premature deaths a year in the UK alone - Skoda and Seat models in the UK were also fitted with the EA189 engine VW used to cheat US emissions tests
Canadian mining company Barrick Gold leaks a million liters of cyanide into river in Argentina
Nigerian troops free 241 women, children in Boko Haram camps, arrest kingpin.
Hajj: 100 dead and 390 injured in Mina stampede
The Associate Press will no longer use the term "skeptic" for those who deny climate change.
German commander in charge of anti-ISIS coalition in Iraq has been found dead in his hotel room in Irbil. "This incident is under investigation and further inform

In [8]:
print(x_input[2])

b'Remember that adorable 9-year-old who sang at the opening ceremonies? That was fake, too.' b"Russia 'ends Georgia operation'" b'"If we had no sexual harassment we would have no children..."' b"Al-Qa'eda is losing support in Iraq because of a brutal crackdown on activities it regards as un-Islamic - including women buying cucumbers" b'Ceasefire in Georgia: Putin Outmaneuvers the West' b'Why Microsoft and Intel tried to kill the XO $100 laptop' b'Stratfor: The Russo-Georgian War and the Balance of Power   ' b"I'm Trying to Get a Sense of This Whole Georgia-Russia War: Vote Up If You Think Georgia Started It, Or Down If you Think Russia Did" b"The US military was surprised by the timing and swiftness of the Russian military's move into South Ossetia and is still trying to sort out what happened, a US defense official said Monday" b'U.S. Beats War Drum as Iran Dumps the Dollar' b'Gorbachev: "Georgian military attacked the South Ossetian capital of Tskhinvali with multiple rocket launcher

In [9]:
# x=(pre_features_GOOG[loc_start_date:-output_time_step])
# print(len(x))
# x_input=[]
# for i in x:
#   sr=""
#   for j in i:
#     sr=sr+j+' '
#   x_input.append(sr)  

In [10]:
 (x_train,  y_train), (x_test, y_test), preproc = text.texts_from_array(x_train=x_input[:-100], y_train=y_input[:-100],
                                                                       x_test=x_input[-100:], y_test=y_input[-100:],
                                                                       class_names=[0,1],
                                                                       preprocess_mode='bert',
                                                                       maxlen=400, 
                                                                       max_features=35000)

downloading pretrained BERT model (uncased_L-12_H-768_A-12.zip)...
[██████████████████████████████████████████████████]
extracting pretrained BERT model...
done.

cleanup downloaded zip...
done.

preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


task: text classification


In [11]:
model = text.text_classifier('bert', train_data=(x_train, y_train), preproc=preproc)
learner = ktrain.get_learner(model, train_data=(x_train, y_train),val_data=(x_test,y_test), batch_size=8)

Is Multi-Label? False
maxlen is 400
done.


In [12]:
learner.fit_onecycle(2e-5,4)



begin training using onecycle policy with max lr of 2e-05...
Epoch 1/4
237/237 [==============================] - 258s 1s/step - loss: 0.7036 - accuracy: 0.5013 - val_loss: 0.6978 - val_accuracy: 0.4500
Epoch 2/4
237/237 [==============================] - 256s 1s/step - loss: 0.6975 - accuracy: 0.5167 - val_loss: 0.6992 - val_accuracy: 0.5700
Epoch 3/4
237/237 [==============================] - 255s 1s/step - loss: 0.6902 - accuracy: 0.5384 - val_loss: 0.6859 - val_accuracy: 0.6000
Epoch 4/4
237/237 [==============================] - 256s 1s/step - loss: 0.6392 - accuracy: 0.6490 - val_loss: 0.7409 - val_accuracy: 0.5000


In [ ]:
learner.validate(val_data=(x_train, y_train), class_names=[0,1])